**DEPRECATED**


In [1]:
# Primary goal of this notebook is to scrape for player data. Trying to find data for many players. 
# Method is scraping leaderboards of (all) countries. This is better than just one/global lb because leaderboards only go up to
# 10k, and we would like more. Then, individually scrape each player's profile for more data.

from ossapi import Ossapi
from ossapi import Cursor
import pycountry
import sqlite3 
import pandas as pd
import plyvel
import pickle
from dataclasses import dataclass
import ossapi.models
import time
from concurrent.futures import ThreadPoolExecutor
import threading
import asyncio
from ossapi import Ossapi


# Step 1: Scrape a list of valid user_ids
Go through leaderboards of all countries to get the most number of unique players. Else, limited to 10k. 
Don't really want to BFS for players, and no other (good) way to get more than 10k players. 

In [2]:
client_id = "28051"
client_secret = "Ry1MCvj2OHJ1mK19M9x1j1sUNFwLQdeJb8Xpfegs"
api = Ossapi(client_id, client_secret)

In [3]:
conn = sqlite3.connect('data/osu_recommender.db')
sql_cursor = conn.cursor()
seen_users = set(sql_cursor.execute("SELECT user_id FROM players").fetchall()) 
to_insert = [] # Cache for batch write optimization. 

def add_player(user_id, pp, seen_users = seen_users, to_insert = to_insert):
    """
    Adds a player to player table in db if not already seen. Only has player and pp information. 
    user_id: integer user_id
    pp: Total pp (float) for that player. 
    seen_users: A set of user_ids already in database/queued for insertion.
    to_insert: A queue of user_ids queued/cached for insertion. 
    """
    if user_id not in seen_users:
        seen_users.add(user_id)
        to_insert.append((user_id, pp))
        
    if len(to_insert) >= 1000:
        sql_cursor.executemany('INSERT INTO players (user_id, pp) VALUES (?, ?)', to_insert)
        to_insert.clear()
        conn.commit()

In [43]:
country_codes = [country.alpha_2 for country in pycountry.countries]
completed_countries = sql_cursor.execute("SELECT country_alpha_2 FROM completed_countries").fetchall()
completed_countries = [country[0] for country in completed_countries]
remaining_countries = [country for country in country_codes if country not in set(completed_countries)]

for country in remaining_countries:
    print(country)
    try:
        lb_cursor = Cursor(page=1)
        while lb_cursor is not None: # Cursor will be none if we have reached the end of the leaderboard
            rankings = api.ranking(mode = "osu", type = "performance", cursor = lb_cursor, country = country)
            lb_cursor = rankings.cursor # Update cursor
            
            for user_stats in rankings.ranking:
                if user_stats.pp < 500:
                    break # Don't add players with less than 500pp. Minor speed up. 
                add_player(user_stats.user.id, user_stats.pp)
    except ValueError as e: # Sometimes the country doesn't have a leader board. (I think 6 of 249)
        print(e)
        print(f'Country {country} doesnt exist (probably). Skipping, and not updating completed_countries')
        continue        
    except KeyboardInterrupt as e:
        print(e)
        print('You keyboard interrupted. Saving progress and exiting.')
        # Save progress and exit
        sql_cursor.executemany('INSERT INTO players (user_id, pp) VALUES (?, ?)', to_insert)
        conn.commit()
        to_insert.clear()
        raise e
    except Exception as e:
        print(e)
        print('uhhhhh')
        raise e
    
    # Add country to completed countries
    sql_cursor.execute(f'INSERT INTO completed_countries (country_alpha_2) VALUES ("{country}")')
    conn.commit()
            

AQ
Expecting value: line 1 column 1 (char 0)
Country AQ doesnt exist (probably). Skipping, and not updating completed_countries
TF
Expecting value: line 1 column 1 (char 0)
Country TF doesnt exist (probably). Skipping, and not updating completed_countries
EH
Expecting value: line 1 column 1 (char 0)
Country EH doesnt exist (probably). Skipping, and not updating completed_countries
HM
Expecting value: line 1 column 1 (char 0)
Country HM doesnt exist (probably). Skipping, and not updating completed_countries
KP
Expecting value: line 1 column 1 (char 0)
Country KP doesnt exist (probably). Skipping, and not updating completed_countries
GS
Expecting value: line 1 column 1 (char 0)
Country GS doesnt exist (probably). Skipping, and not updating completed_countries
SH
Expecting value: line 1 column 1 (char 0)
Country SH doesnt exist (probably). Skipping, and not updating completed_countries
SS
Expecting value: line 1 column 1 (char 0)
Country SS doesnt exist (probably). Skipping, and not updat

In [12]:

# user_ids = players['user_id'].values
# user_id = user_ids[0]
# user = api.user(user_id, mode ='osu', key='id') 
# user_scores = api.user_scores(user_id, type='best', mode='osu', limit=100)
# user_beatmaps = [user_score.beatmap.id  for user_score in user_scores]
# user_beatmaps

# # Temporarily work on backup db

# user_ids = cursor.execute('SELECT user_id FROM players').fetchall()
# user_ids = [user_id[0] for user_id in user_ids]
# for user_id in user_ids:
#     user = api.user(user_id, mode ='osu', key='id')
#     user_scores = api.user_scores(user_id, type='best', mode='osu', limit=100)
#     user_beatmaps = [user_score.beatmap.id  for user_score in user_scores]
#     # Would like to keep one row for each.... Unless we can do some cassandra shit. 

[2983479,
 3963421,
 4033854,
 3962022,
 3967160,
 3665005,
 3355394,
 2111505,
 3873445,
 2684029,
 2597236,
 1651290,
 3906726,
 2659369,
 2567509,
 1632673,
 898576,
 931597,
 2902017,
 2570594,
 904687,
 1938365,
 713442,
 2330357,
 738063,
 1100265,
 2444149,
 2316176,
 2185824,
 827803,
 3874766,
 1194389,
 1572868,
 1621068,
 2419085,
 3933758,
 1610745,
 1619555,
 1702172,
 930249,
 1706834,
 735272,
 1829998,
 723891,
 2469345,
 1848919,
 1483372,
 827488,
 2245766,
 760488,
 2166311,
 1924025,
 2415690,
 2245774,
 884664,
 3069848,
 1643270,
 1726474,
 1612454,
 1165064,
 933875,
 1655981,
 2237470,
 1716732,
 147327,
 1744385,
 3561624,
 1033739,
 1849120,
 2615748,
 2034983,
 2036365,
 2570165,
 1018869,
 3628459,
 3853713,
 2778999,
 602259,
 1645698,
 1827323,
 2887834,
 2331417,
 1897995,
 2246465,
 1186443,
 1023307,
 2606880,
 2168425,
 2872154,
 1828339,
 2510778,
 2025938,
 1588369,
 1032126,
 1642274,
 3746239,
 2059455,
 2067426,
 1897577,
 2826730]

# Step 2: Get user top play data. 
 Realize that relational database has no like benefits.... In fact, going to migrate away to a levelDB because I only need key:val pairs, and I would like to be able to store (byte) objects. Furthermore, stoarge is becoming a concern. 400,000 users, with 100 top plays each. Each play being 1KB? 40GB of data. Not impossible, but I value my storage space.  

## Pickle can't serialize functions in User class.

### Define Custom Classes for serialization and storage purposes. 

In [9]:
@dataclass
class Beatmap:
    """
    Keeps select attributes from ossapi.models.Beatmap
    """
    def __init__(self, beatmap: ossapi.models.Beatmap):
        self.accuracy = beatmap.accuracy
        self.ar = beatmap.ar
        self.beatmapset_id = beatmap.beatmapset_id
        self.bpm = beatmap.bpm
        self.cs = beatmap.cs
        self.count_circles = beatmap.count_circles
        self.count_sliders = beatmap.count_sliders
        self.count_spinners = beatmap.count_spinners
        self.difficulty_rating = beatmap.difficulty_rating
        self.beatmap_id = beatmap.id
        self.max_combo = beatmap.max_combo
        self.drain = beatmap.drain
        
        
@dataclass
class Score:
    """
    Keeps select attributes from ossapi.models.Score
    """
    def __init__(self, score: ossapi.models.Score):
        self.accuracy = score.accuracy
        self.score_id = score.id # 
        self.pp = score.pp
        self.perfect = score.perfect
        self.mods = score.mods.value # Bitwise enumeration of mods. https://github.com/ppy/osu-api/wiki#mods
        self.beatmap = Beatmap(score.beatmap)
        
@dataclass
class UserTopScores:
    """
    Class for tracking an osu player's top scores. Similar to return type of ossapi.user_scores() = list[Score]. 
    Keeps only select attributes for storage and pickling purposes. 
    """
    def __init__(self, user_id, top_scores):
        self.user_id = user_id
        self.scores = []
        top_scores
        for score in top_scores:
            self.scores.append(Score(score)) 
            
async def pull_top_scores(api, user_id):
    """
    Asynchronously pulls top scores from asynch ossapi
    """
    top_scores = await api.user_scores(user_id, type = 'best', mode = 'osu', limit = 100)
    return (user_id, top_scores)
    


## Scrape data

### Manage DB's

In [8]:
try:
    playerDB.close()
except Exception as e:
    print(e)
playerDB = plyvel.DB('data/playerDB', create_if_missing=True)


try:
    conn.close()
except:
    print(e)
sql_conn = sqlite3.connect('data/osu_recommender.db')

# Need async api for following
async_api = Ossapi(client_id, client_secret)

### Iterate over user_ids.
Networked I/O blocked. Need to multi-thread. 

In [14]:
lock = threading.Lock()
num_done = 0
last_time = time.time()
def scrape_top_scores(user_ids_subset, playerDB = playerDB):
    """
    Work function for multithreading scraping of user_id top scores.
    Not going to lock because shouldn't be possible to have concurrent accesses to same key. Terrible programming practice though. 
    """
    global num_done
    global last_time
    for user_id in user_ids_subset:
        ser_user_id = pickle.dumps(user_id)
        if playerDB.get(ser_user_id) is None: 
            user_top_scores =  UserTopScores(user_id, async_api)
            ser_user_top_scores = pickle.dumps(user_top_scores)
            playerDB.put(ser_user_id, ser_user_top_scores)
    
        num_done += 1
        if num_done % 100 == 0:
            print(str(num_done) + ": " + str(time.time() - last_time))
            last_time = time.time()

# Need to partition user_ids so that each thread does different keys. 
user_ids = pd.read_sql_query("SELECT user_id FROM players", sql_conn)
user_ids = list(user_ids['user_id'])
num_partitions = 5
size_partition = len(user_ids) // num_partitions
p_user_ids = [user_ids[i:i + size_partition] for i in range(0, (len(user_ids)//num_partitions)*num_partitions, size_partition)]
if len(user_ids) % num_partitions != 0:
    p_user_ids[-1].extend(user_ids[(len(user_ids)//num_partitions)*num_partitions:])
    
with ThreadPoolExecutor(max_workers=num_partitions) as executor:
    for user_ids in p_user_ids:
        executor.submit(scrape_top_scores, user_ids)
        



100: 0.23535633087158203
200: 0.002130270004272461
300: 0.0014386177062988281
400: 0.0018777847290039062
500: 0.0018413066864013672
600: 0.0013065338134765625
700: 0.002015829086303711
800: 0.0017189979553222656
900: 0.0014846324920654297
1000: 0.0015342235565185547
1100: 0.0017323493957519531
1200: 0.002088785171508789
1300: 0.0018765926361083984
1400: 0.0017540454864501953
1500: 0.002164602279663086
1600: 0.0019135475158691406
1700: 0.0017082691192626953
1800: 0.001783132553100586
1900: 0.002758502960205078
2000: 0.001947164535522461
2100: 0.0033960342407226562
2200: 0.0053806304931640625
2300: 0.006060600280761719
2400: 0.005263328552246094
2500: 0.0049021244049072266
2600: 0.004897117614746094
2700: 0.005650520324707031
2800: 0.005185604095458984
2900: 0.00496673583984375
3000: 0.004995584487915039
3100: 0.005558013916015625
3200: 0.0050792694091796875
3300: 0.0049936771392822266
3400: 0.005563020706176758
3500: 0.005673885345458984
3600: 0.005368709564208984
3700: 0.00502014160156